In [2]:
import pandas as pd
import swifter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 5]


In [3]:
df = pd.read_csv("./Data/Part 1 - Telemarketing.csv")